In [1]:
# Requests 및 데이터 핸들링 관련 import
import requests # HTTP 요청을 보내기 위한 라이브러리
import pandas as pd # 데이터프레임(DataFrame) 처리 라이브러리
import random # 무작위 선택을 위한 라이브러리

# PyTorch 및 BERT 모델 관련 import
import torch # PyTorch 텐서 및 디바이스(CPU/GPU) 제어 라이브러리
from transformers import AutoTokenizer, AutoModel # Huggingface에서 사전 학습된 BERT 모델과 토크나이저 로드

# Paraphrase 유틸 import
from paraphrase_util import build_paraphrase_candidates # 증상 paraphrase 후보 추천 함수

# ------------------------- #
# 1. 모델 로딩
# ------------------------- #
model_name = "madatnlp/km-bert" # 사용할 KM-BERT 모델 이름 설정
tokenizer = AutoTokenizer.from_pretrained(model_name) # KM-BERT 토크나이저 불러오기
bert_model = AutoModel.from_pretrained(model_name) # KM-BERT 모델 불러오기
bert_model.eval() # 모델을 평가 모드로 전환 (학습 X, 추론 전용)

# CUDA (GPU) 사용 가능 여부에 따라 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_model.to(device) # 모델을 설정한 디바이스로 이동

# ------------------------- #
# 2. 데이터 로딩 함수
# ------------------------- #
def load_data(
    disease_url, # 질병 데이터를 가져올 API URL
    symptom_url # 증상 데이터를 가져올 API URL
):
    disease_response = requests.get(disease_url) # 질병 데이터 요청
    symptom_response = requests.get(symptom_url) # 증상 데이터 요청
    disease_response.raise_for_status() # 질병 데이터 요청 실패 시 예외 발생
    symptom_response.raise_for_status() # 증상 데이터 요청 실패 시 예외 발생

    # 질병 데이터 content 부분 추출
    diseases = disease_response.json()['content']
    
    # 증상 데이터 content 부분 추출
    symptoms = symptom_response.json()['content']
    
    return diseases, symptoms # 질병 리스트와 증상 리스트 반환

# ------------------------- #
# 3. 증상 ID → 이름 매핑 함수
# ------------------------- #
def map_symptom_ids_to_names(
    symptoms # 증상 데이터 리스트
):
    # 증상 ID를 키로, 증상 이름을 값으로 매핑한 딕셔너리 반환
    return { symptom['id']: symptom['name'] for symptom in symptoms }

# ------------------------- #
# 4. 문장 생성 함수
# ------------------------- #
def generate_sentence(
    disease_name, # 질병 이름 (ex: "감기")
    symptom_names # 증상 이름 리스트 (ex: ["목 통증",  "기침", "열감"])
):
    if not symptom_names: # 증상 리스트가 비어있는 경우
        return None # None 반환

    # 증상 리스트 중 3~4개를 무작위로 선택
    selected_symptoms = random.sample(symptom_names, min(len(symptom_names), random.choice([3, 4])))

    # 증상들을 이어붙이기
    if len(selected_symptoms) > 1: # 증상이 두 개 이상인 경우
        symptom_text = ", ".join(selected_symptoms[:-1]) + " 그리고 " + selected_symptoms[-1]
    else: # 증상이 한 개인 경우
        symptom_text = selected_symptoms[0]

    # 다양한 문장 템플릿 중 하나를 랜덤으로 선택
    templates = [
        f"{symptom_text} 증상이 나타나고 있어요. 혹시 {disease_name}일까요?",
        f"{symptom_text}로 고생하고 있어요. {disease_name} 가능성이 있을까요?",
        f"최근 {symptom_text}가(이) 심해졌어요. {disease_name}인가요?",
        f"몸에 {symptom_text} 같은 증상이 나타났습니다. {disease_name}인지 걱정돼요.",
        f"{symptom_text} 때문에 일상생활이 힘들어요. {disease_name}을 의심해볼 수 있나요?",
    ]

    # 생성된 문장 중 하나를 랜덤 반환
    return random.choice(templates)

# ------------------------- #
# 5. 전체 데이터셋 생성 함수
# ------------------------- #
def generate_dataset(
    diseases, # 질병 데이터 리스트
    symptom_id_to_name, # 증상 id를 증상 이름으로 매핑한 딕셔너리
    paraphrase_map # 증상 이름을 paraphrase 후보로 매핑한 딕셔너리
):
    # 생성된 문장 데이터를 저장할 리스트
    generated_data = []

    for disease in diseases:
        disease_id = disease['id'] # 질병 ID
        disease_name = disease['name'] # 질병 이름
        symptom_ids = disease['symptoms'] # 연결된 증상 ID 목록

        # 증상 ID 리스트를 증상 이름 리스트로 변환
        symptom_names = [symptom_id_to_name.get(id, f"알수없는증상({id})") for id in symptom_ids]

        # paraphrase 후보 적용
        paraphrased_symptoms = []
        for symptom in symptom_names:
            candidates = paraphrase_map.get(symptom, [symptom]) # 후보가 없으면 원본 사용
            selected = random.choice(candidates) # 후보 중 하나 무작위 선택
            paraphrased_symptoms.append(selected)

        if paraphrased_symptoms: # 증상 이름 리스트가 비어있지 않은 경우
            sentence = generate_sentence(disease_name, paraphrased_symptoms) # 질병명과 증상명으로 생성
            if sentence: # 문장을 성공적으로 생성한 경우
                generated_data.append({
                    'disease_id': disease_id, # 질병 ID 저장
                    'disease_name': disease_name, # 질병 이름 저장
                    'generated_sentence': sentence # 생성된 문장 저장
                })

    # 결과를 Pandas DataFrame으로 반환
    return pd.DataFrame(generated_data)

# ------------------------- #
# 6. CSV 저장 함수
# ------------------------- #
def save_to_csv(
    df, # 저장할 데이터프레임
    filename # 저장할 파일 이름
):
    # CSV 파일로 저장 (UTF-8 BOM 포함 인코딩)
    df.to_csv(filename, index=False, encoding='utf-8-sig')

# ------------------------- #
# 7. Paraphrase Map 저장 함수
# ------------------------- #
def save_paraphrase_map_to_csv(
    paraphrase_map, # 원본 증상명 → paraphrase 후보 리스트 매핑 딕셔너리
    filename # 저장할 파일 이름
):
    # CSV 저장용 데이터 변환
    rows = []
    for symptom, candidates in paraphrase_map.items():
        rows.append({
            'original_symptom': symptom,
            'paraphrase_candidates': ", ".join(candidates)
        })
    df = pd.DataFrame(rows)
    df.to_csv(filename, index=False, encoding='utf-8-sig')

# ------------------------- #
# 메인 실행 흐름
# ------------------------- #
if __name__ == "__main__":
    # 질병 데이터 API URL
    DISEASE_API_URL = "http://localhost:8080/api/diseases/processed?page=0&size=500"
    # 증상 데이터 API URL
    SYMPTOM_API_URL = "http://localhost:8080/api/symptoms?page=0&size=3000"

    # 질병 및 증상 데이터 로딩
    diseases, symptoms = load_data(DISEASE_API_URL, SYMPTOM_API_URL)

    # 증상 ID → 이름 매핑 생성
    symptom_id_to_name = map_symptom_ids_to_names(symptoms)

    # 전체 증상 이름 리스트 생성
    all_symptom_names = list(symptom_id_to_name.values())

    # 전체 증상 이름 기반 paraphrase 후보 생성
    paraphrase_map = build_paraphrase_candidates(all_symptom_names, similarity_threshold=0.8)

    # 질병-증상 기반 문장 데이터셋 생성
    generated_df = generate_dataset(diseases, symptom_id_to_name, paraphrase_map)

    # paraphrase map을 CSV 파일로 저장
    save_paraphrase_map_to_csv(paraphrase_map, 'paraphrase_map.csv')

    # 생성된 데이터셋을 CSV 파일로 저장
    save_to_csv(generated_df, 'generated_disease_sentences_v2.csv')

c:\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
